# Дейкстра

### Формуирем поле (таблицу) со случайными значениями

In [62]:
import random

rows = 30 # кол-во строк поля
cols = 30 # кол-во столбцов поля

# создается поле размером rows X cols
field = [[random.randint(1,9) for i in range(0,cols)] for x in range(0,rows)]


### Преобразуем поле (таблицу в словарь)

In [63]:
def make_graph_dict(lst):
    '''формируем граф из таблицы со связями в 4 направлениях
    Граф - словарь, где ключами являются наименования узлов, 
    а значения - тоже словари из узлов-последователей и веса ребра до каждого из них
    '''
    graph = {}

    for row in range(len(lst)): # формируем узлы
        for col in range(len(lst[0])):
            graph[f'{row},{col}'] = {}

    for each in graph: 
        r,c = map(int,each.split(','))
        if r+1 <= rows-1: #ячейка снизу
            graph[each][f'{r+1},{c}']= lst[r+1][c]
        if c+1 <= cols-1: #ячейка справа
            graph[each][f'{r},{c+1}']= lst[r][c+1]
        if r-1 >= 0:      #ячейка сверху
            graph[each][f'{r-1},{c}']= lst[r-1][c]
        if c-1 >= 0:      #ячейка слева
            graph[each][f'{r},{c-1}']= lst[r][c-1]
    return graph
    
graph = make_graph_dict(field) #получили граф из таблицы


### Определяем класс

In [64]:
class Graph:
    '''
    costs - словарь стоимостей для каждого узла
    end - наименование узла-цели
    start - наименование узла-начала
    parents - словарь родителей для узла (по нему восстанавливается путь)
    node_amt - кол-во узлов графа

    dijkstra - (self,start,end,start_val=0) определения кратчайего пути от start до end
    '''

    def __init__(self,graph):
        self.graph = graph
        self.node_amt = len(graph)
        self.start = 'ND'
        self.end = 'ND'
        self.parents = 'ND'
        self.costs = 'ND'
        self.processed = 'ND'

    def _find_lowest_cost_node(self):
        '''выбираем непосещенный узел с минимальной стоимостью, от него будем пересчитывать стоимости последователей '''
        lcn_val = float('inf')
        lcn = None
        for node in self.costs:
            if self.costs[node] < lcn_val and node not in self.processed:
                lcn_val = self.costs[node]
                lcn = node
        return lcn
    
    
    def dijkstra(self,start,end,start_val=0):
        '''Необходимо передать граф (в формате узел - его последователи с ребрами), стартовая стоимость, стартовый и конечные узлы
        1 - costs создаем стоимости для всех узлов (равны бесконечности)
        2 - создаем processed - обработанные узлы, чтобы они не попадались при выборе lowest_cost_node
        3 - parents - словарь для записи пути'''
        self.start = start
        self.end = end

        self.costs = {vertex: float('inf') for vertex in self.graph} # создаем словарь со стоимостями (бесконечность для всех)
        self.costs[start] = start_val # для начальной точки стоимость
        
        self.processed = [] # список обработанных узлов (старт уже включен)
        self.parents = dict()
        
        lowest_cost_node = self._find_lowest_cost_node() #определяем самый дешевый узел из доступных непосещенных
        while lowest_cost_node != end: 
            for node in self.graph[lowest_cost_node]: #для каждого последователя самого дешевого узла
                if self.costs[node] > self.graph[lowest_cost_node][node] + self.costs[lowest_cost_node]: #если стоимость самого дешевого + ребро до последователя меньше записанной стоимости последователя, то перезаписываем
                    self.costs[node] = self.graph[lowest_cost_node][node] + self.costs[lowest_cost_node]
                    self.parents[node] = lowest_cost_node
                    
            self.processed.append(lowest_cost_node) # добавляем самый дешевый узел в пройденные
            lowest_cost_node = self._find_lowest_cost_node() #обновляем значение самого дешевого узла при шаге вперед из доступных непройденных
        
        for node in self.graph[lowest_cost_node]: #для каждого последователя самого дешевого узла
                if self.costs[node] > self.graph[lowest_cost_node][node] + self.costs[lowest_cost_node]: #если стоимость самого дешевого + ребро до последователя меньше записанной стоимости последователя, то перезаписываем
                    self.costs[node] = self.graph[lowest_cost_node][node] + self.costs[lowest_cost_node]
        
        self.shortest_path_cost = self.costs[self.end]
        self.shortest_path = self.make_shortest_path_list()

        print(f'Маршрут от "{self.start}" до "{self.end}" просчитан ({self.shortest_path_cost})')

    def make_shortest_path_list(self):
        cur_node = self.end
        shortest_path = [cur_node]
        while self.start != cur_node:
            shortest_path.append(self.parents[cur_node])
            cur_node=self.parents[cur_node]
        shortest_path = shortest_path[::-1]
        return shortest_path
    
    def display_shortest_path(self):
        i=0
        for each in self.shortest_path:
            i+=1
            if i%9 == 0:
                print(f'|{each}|',end=' ->\n')
            elif self.end == each:
                print(f'|{each}|')
            else:
                print(f'|{each}|',end=' -> ')

        

### Пример работы

In [65]:
G = Graph(graph)
G.dijkstra(
    start='0,0', #левый верхний угол сгенерированного поля
    end=f'{rows-1},{cols-1}', #нижний правый угол сгенерированного поля
    start_val=field[0][0] #значение в левом верхнем углу
    )
G.display_shortest_path() # выводим кратчайший путь

Маршрут от "0,0" до "29,29" просчитан (177)
|0,0| -> |0,1| -> |1,1| -> |2,1| -> |3,1| -> |3,2| -> |4,2| -> |5,2| -> |6,2| ->
|7,2| -> |8,2| -> |8,3| -> |8,4| -> |8,5| -> |7,5| -> |7,6| -> |7,7| -> |7,8| ->
|7,9| -> |7,10| -> |8,10| -> |8,11| -> |9,11| -> |9,12| -> |9,13| -> |9,14| -> |9,15| ->
|10,15| -> |10,16| -> |10,17| -> |10,18| -> |11,18| -> |12,18| -> |12,19| -> |12,20| -> |12,21| ->
|13,21| -> |13,22| -> |13,23| -> |13,24| -> |14,24| -> |15,24| -> |16,24| -> |16,25| -> |17,25| ->
|17,26| -> |18,26| -> |19,26| -> |20,26| -> |21,26| -> |22,26| -> |22,27| -> |23,27| -> |24,27| ->
|25,27| -> |26,27| -> |27,27| -> |27,28| -> |28,28| -> |28,29| -> |29,29|


### Код для визуалиации прохождения поля

In [66]:
# ОТОБРАЖЕНИЕ ПОЛЯ
for each in field:
    for every in each:
        print(every,end=' '*(4 - len(str(every))))
    print()

9   8   3   6   7   7   4   7   3   3   4   2   1   9   2   9   7   9   9   5   7   9   2   1   5   3   9   2   6   7   
9   2   6   8   6   9   8   1   9   6   1   5   2   4   3   1   7   9   6   6   6   8   6   6   7   5   3   9   2   9   
9   1   7   6   7   6   6   3   7   6   4   7   1   1   3   1   5   3   9   1   8   7   6   9   7   3   9   1   8   3   
4   1   2   8   3   7   4   5   4   9   5   5   2   3   4   8   5   9   4   1   4   1   2   5   4   3   2   7   8   3   
5   6   3   9   2   4   7   5   9   4   5   8   2   1   3   7   1   9   1   3   1   1   7   1   1   6   9   6   9   8   
1   3   7   1   8   2   8   8   5   3   7   1   6   2   5   2   9   3   9   1   2   9   6   8   7   8   8   6   1   7   
5   6   1   4   7   9   8   7   7   3   1   7   8   6   4   9   9   5   2   1   3   1   9   4   6   6   6   3   4   5   
5   4   1   3   9   4   2   9   1   2   3   8   7   1   8   2   7   7   3   8   7   4   6   8   1   7   7   4   2   6   
9   5   1   2   1   2   7   9   

In [67]:
# ОТОБРАЖЕНИЕ ТОЛЬКО ПУТИ
from copy import deepcopy
field2 = deepcopy(field)
for r_ind,r in enumerate(field2):
    for c_ind,c in enumerate(r):
        if f'{r_ind},{c_ind}' in G.shortest_path:
            field2[r_ind][c_ind] = c
        else:
            field2[r_ind][c_ind] = ' '

for each in field2:
    for every in each:
        print(every,end=' '*(4 - len(str(every))))
    print()

9   8                                                                                                                   
    2                                                                                                                   
    1                                                                                                                   
    1   2                                                                                                               
        3                                                                                                               
        7                                                                                                               
        1                                                                                                               
        1           4   2   9   1   2   3                                                                               
        1   2   1   2           

In [68]:
val = 0
for r_ind,r in enumerate(field2):
    for c_ind,c in enumerate(r):
        if f'{r_ind},{c_ind}' in G.shortest_path:
            field2[r_ind][c_ind] = c
        else:
            field2[r_ind][c_ind] = ' '

for each in field2:
    for every in each:
        if isinstance(every,int):
            val+=every
            print(val,end=' '*(4 - len(str(val))))
        else:
            print(every,end=' '*3)
    print()

9   17                                                                                                                  
    19                                                                                                                  
    20                                                                                                                  
    21  23                                                                                                              
        26                                                                                                              
        33                                                                                                              
        34                                                                                                              
        35          39  41  50  51  53  56                                                                              
        57  59  60  62          